In [1]:
from transformers import AutoModel, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-roberta-wwm-ext")
lawformer = AutoModel.from_pretrained("thunlp/Lawformer")

d:\programs\anaconda\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at thunlp/Lawformer were not used when initializing LongformerModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Som

In [2]:
import torch
import json
class CailDataset(torch.utils.data.Dataset):
    def __init__(self, path):
        self.path = path 
        file = open(path,'r',encoding='utf-8')
        cases = file.readlines()
        self.cases = [json.loads(i) for i in cases]
        self.length = len(self.cases)
    def __len__(self):
        return self.length

    def __getitem__(self, index):
        data = self.cases[index]
        caseA = "".join(data['Case_A'])
        caseB = "".join(data['Case_B'])
        label = data['label']
        inputs = tokenizer([caseA,caseB], return_tensors="pt",padding="max_length",max_length=300,truncation=True)
        return inputs,label



In [19]:
dataset = CailDataset('./cail/competition_stage_2_train.json')
def collate_fn(data):
    tensors = [i[0] for i in data]
    labels = [i[1]  for i in data]
    inputs_ids = torch.cat([i['input_ids'] for i in tensors])
    token_type_ids = torch.cat([i['token_type_ids'] for i in tensors])
    mask = torch.cat([i['attention_mask'] for i in tensors])
    inputs = {"input_ids":inputs_ids,"token_type_ids":token_type_ids,"attention_mask":mask}
    return inputs,labels
#数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=3,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)

# 查看数据样例
for i, batch in enumerate(loader):
    break

#aloader = torch.utils.data.DataLoader(dataset=dataset,batch_size=3,shuffle=False,drop_last=True)
# for i,abatch in enumerate(aloader):
#     break
def manifest(name,batch):
    print(name)
    print(type(batch))# list 
    print(len(batch))# ==batch_size
    print(type(batch[0])) # tuple = dictionary of data + label
    print(type(batch[0][0]),type(batch[0][1])) # dict and int 
    for i in batch[0][0]:
        print(i,batch[0][0][i].shape)
    print("end")

# manifest("with collate_fn",batch)
#manifest("without collate_fn",abatch)
print("without collate ")
# print(type(abatch),len(abatch))
# print(type(abatch[0]),type(abatch[1]))
# for key in abatch[0]:
#     print(key,abatch[0][key].shape)
# print(abatch[1])
# print(len(loader))
# print(tokenizer.decode(inputs['input_ids'][0][1]))
# # print(labels[0])
#print(len(batch),len(batch[0]),len(batch[0][0]))
# for k, v in inputs.items():
#     print(k, v.shape)
# inputs, labels = dataset[0]

# len(dataset), inputs['input_ids'].shape, labels
batch

without collate 


({'input_ids': tensor([[ 101, 5307, 2144,  ..., 6158, 1440,  102],
          [ 101, 5307, 2144,  ..., 4638, 1297,  102],
          [ 101, 5307, 2144,  ..., 1440,  782,  102],
          [ 101, 1062, 6401,  ...,  809, 2807,  102],
          [ 101, 5307, 2144,  ..., 8790, 5439,  102],
          [ 101, 5307, 2144,  ..., 6395,  816,  102]]),
  'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]]),
  'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1]])},
 [0, 1, 0])

In [4]:
from time import time
time1 = time()
for b in batch:
    # print(type(b[0]))
    # for key in b[0]:
    #     print(key,b[0][key].shape)
    lawformer(**b[0])
time2 = time()

# print(tensorcopy.shape)
# print(tensorcopy2.shape)
# print(tensorcopy3.shape)
time3 = time()


In [5]:
print(time2-time1,time3-time2)

10.718900918960571 10.254740953445435


In [ ]:
trainset = CailDataset('./cail/competition_stage_2_train.json')
testset = CailDataset('./cail/competition_stage_2_test.json')
#数据加载器
trainloader = torch.utils.data.DataLoader(dataset=trainset,
                                     batch_size=1,
                                     shuffle=True,
                                     drop_last=True)
testloader = torch.utils.data.DataLoader(dataset=testset,
                                     batch_size=16,
                                     shuffle=False,
                                     drop_last=True)

In [21]:

#模型试算
#[b, lens] -> [b, lens, 768]
#pretrained(**inputs).last_hidden_state.shape
import torch  
class Lawfomer2Super(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.tuneing = False
        self.pretrained = lawformer
        # 300 is max length of lawformer 
        self.conv1d = torch.nn.Conv1d(in_channels=300,out_channels=1,kernel_size=1)
        self.conv2d  = torch.nn.Conv2d(in_channels = 1, out_channels = 2, kernel_size=(2,1),stride=(2,1))
        self.fc = torch.nn.Linear(768, 3)

    def forward(self, inputs):
        #print(inputs.keys())
        # inputs : batch_size * max_length * 768 
        # conv1d output : batch_size * 1 * 768
        # result : batch_size*1 * 3 
        if self.tuneing:
            #print("here")
            out = self.pretrained(**inputs).last_hidden_state
        else:
            with torch.no_grad():
                out = self.pretrained(**inputs).last_hidden_state
        out = self.conv1d(out)
        out = self.conv2d(out)
        out = self.fc(out)
        return out

    def fine_tuneing(self, tuneing):
        self.tuneing = tuneing
        if tuneing:
            for i in self.pretrained.parameters():
                i.requires_grad = True

            lawformer.train()
            self.pretrained = lawformer
        else:
            for i in lawformer.parameters():
                i.requires_grad_(False)

            lawformer.eval()
            print("any")
            self.pretrained = None

In [22]:
lf2classify = Lawfomer2Super()
lf2classify.fine_tuneing(tuneing=True)

In [23]:
#inputs = {"input_ids":tensorcopy[:4],"token_type_ids":tensorcopy2[:4],"attention_mask":tensorcopy3[:4]}
lf2classify(batch).shape

torch.Size([6, 300, 768])
torch.Size([6, 1, 768])


torch.Size([6, 1, 3])

In [ ]:
data,label =trainset[0][0],trainset[0][1]
for i in data:
    print(i,data[i].shape)
lf2classify(data)
trainloader = torch.utils.data.DataLoader(dataset=trainset,
                                     batch_size=1,
                                     shuffle=True,
                                     drop_last=True)
for i , (data1,label1) in enumerate(trainloader):
    break

In [ ]:
for i in data:
    print(i,data1[i].shape,data[i].shape)

In [ ]:
from transformers import AdamW
#训练
from tqdm import tqdm,trange
def train(epochs):
    lr = 1e-3 if lf2classify.tuneing else 5e-4

    #训练
    optimizer = AdamW(lf2classify.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss()

    lf2classify.train()
    for epoch in range(epochs):
        lf2classify.train()
        step = 0 
        #for step, (inputs, labels) in tqdm(enumerate(trainloader)):
        for i in trange(len(trainset.cases)):
            (inputs, labels) = trainset[i]
            step+=1
            #模型计算
            #print(len(inputs),inputs.keys())
            outs = lf2classify(inputs)
            #梯度下降
            outs = outs.squeeze()
            #print(outs,labels)
            one_hot = torch.zeros(3)
            one_hot[labels] = 1
            loss = criterion(outs, torch.FloatTensor(one_hot))
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            if step % 50 == 0:
                print("in epoch "+str(epoch)+" current train loss is "+str(loss.item()))
        if epoch%10==0 or epoch==epochs-1:
            lf2classify.eval()
            correct = 0 
            with torch.no_grad:
                for step, (inputs, labels) in enumerate(testloader):
                    outs = lf2classify(inputs)
                    fakes = torch.argmax(outs)
                    correct+=sum(fakes==labels)
            
            print("in epoch "+str(epoch)+" current validation accuracy is "+str(correct/len(testloader)))

        torch.save(lf2classify, 'model/lawformer_微调.model')


print(sum(p.numel() for p in lf2classify.parameters()))

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
criterion(torch.FloatTensor(3),torch.zeros(3))
train(1)

In [ ]:
#lawformer(**inputs).last_hidden_state.shape
train(1)

In [ ]:
for i in inputs:
    print(inputs[i].shape)
lf2classify(inputs).shape